In [ ]:
from fairlearn.metrics import *
import pandas as pd
from sklearn.metrics import accuracy_score

test_df = pd.read_csv("test_set_with_predictions.csv")
y_test_binary = test_df['credit_status']
y_pred_custom = test_df['predicted_credit_status']

# Creating the 'age_bin' column with the adjusted bins for scaled age
test_df['age_bin'] = pd.cut(
    test_df['age'],  
    bins=[0, 0.306122, 0.612244, 1],  # Adjusted bins for scaled age (corresponding to original age ranges)
    labels=['<35', '35-50', '>50'],
    right=False  # Right edge is excluded, i.e., <30 will not include 30
)

# Fairness metrics by gender
fairness_rf_gender = MetricFrame(
    metrics={
        "Accuracy": accuracy_score,
        "TPR": true_positive_rate,
        "FPR": false_positive_rate
    },
    y_true=y_test_binary,
    y_pred=y_pred_custom,
    sensitive_features=test_df['gender']
)
print("🎯 Random Forest Fairness Metrics by Gender:\n", fairness_rf_gender.by_group)

# Fairness metrics by age group
fairness_rf_age = MetricFrame(
    metrics={
        "Accuracy": accuracy_score,
        "TPR": true_positive_rate,
        "FPR": false_positive_rate
    },
    y_true=y_test_binary,
    y_pred=y_pred_custom,
    sensitive_features=test_df['age_bin']
)
print("\n🎯 Random Forest Fairness Metrics by Age Group:\n", fairness_rf_age.by_group)

# Disparity metrics (difference across groups)
dpd_rf_gender = demographic_parity_difference(y_test_binary, y_pred_custom, sensitive_features=test_df['gender'])
eod_rf_gender = equalized_odds_difference(y_test_binary, y_pred_custom, sensitive_features=test_df['gender'])

dpd_rf_age = demographic_parity_difference(y_test_binary, y_pred_custom, sensitive_features=test_df['age_bin'])
eod_rf_age = equalized_odds_difference(y_test_binary, y_pred_custom, sensitive_features=test_df['age_bin'])

print(f"\n📊 Demographic Parity Difference (Gender): {dpd_rf_gender:.4f}")
print(f"📊 Equalized Odds Difference (Gender): {eod_rf_gender:.4f}")

print(f"\n📊 Demographic Parity Difference (Age Group): {dpd_rf_age:.4f}")
print(f"📊 Equalized Odds Difference (Age Group): {eod_rf_age:.4f}")


🎯 Random Forest Fairness Metrics by Gender:
         Accuracy       TPR       FPR
gender                              
0       0.814565  0.491647  0.155253
1       0.818410  0.531532  0.152214

🎯 Random Forest Fairness Metrics by Age Group:
          Accuracy       TPR       FPR
age_bin                              
35-50    0.786301  0.481203  0.183120
<35      0.858847  0.500000  0.113904
>50      0.815678  0.536481  0.153738

📊 Demographic Parity Difference (Gender): 0.0034
📊 Equalized Odds Difference (Gender): 0.0399

📊 Demographic Parity Difference (Age Group): 0.0691
📊 Equalized Odds Difference (Age Group): 0.0692
